# Deep Q-Learning for Lunar Landing

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [ ]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


### Importing the libraries

In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [ ]:
class Network(nn.Module):

  def __init__(self, state_size, action_size, seed = 42):  #state_size- no of inputs in a state(8)
    super(Network, self).__init__()
    self.seed = torch.manual_seed(seed)
    #Input layer to first layer:
    self.fc1 = nn.Linear(state_size, 64) #64- optimal number of neurons in first layer of NN (after trial and error)
    self.fc2 = nn.Linear(64, 64)  #First layer to 2nd layer
    self.fc3 = nn.Linear(64, action_size) #2nd layer to output layer

  def forward(self, state): #propogates signal forward
    x = self.fc1(state)
    x = F.relu(x) #reLu activation function at each layer
    x = self.fc2(x)
    x = F.relu(x)
    return self.fc3(x)

## Part 2 - Training the AI

### Setting up the environment

In [ ]:
import gymnasium as gym
env = gym.make('LunarLander-v3') # The Lunar Lander environment was upgraded to v3
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n
print('State shape: ', state_shape)
print('State size: ', state_size)
print('Number of actions: ', number_actions)

State shape:  (8,)
State size:  8
Number of actions:  4


### Initializing the hyperparameters

In [ ]:
learning_rate = 5e-4
minibatch_size = 100
discount_factor = 0.99 #further away from goal, more discounted value of state- hence discount factor must be high
replay_buffer_size = int(1e5) #100000
interpolation_parameter = 1e-3 #after experiment-optimal

### Implementing Experience Replay

    # Each experience has 4 elements- state, action, reward, nextstate and done(boolean whether done or not with that specific episode)

In [ ]:
class ReplayMemory(object):

  def __init__(self, capacity): #capacity of memory
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.capacity = capacity
    self.memory = []

  #add those 100000 experiences in the replay memory buffer
  def push(self, event):
    self.memory.append(event)
    if len(self.memory) > self.capacity:
      del self.memory[0]  #delete the oldest experience

  #randomly select batch of experiences from memory buffer
  def sample(self, batch_size):
    experiences = random.sample(self.memory, k = batch_size)
     #Next we need to get each element of each experience and stack them

    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
    return states, next_states, actions, rewards, dones

### Implementing the DQN class

In [ ]:
class Agent():

  def __init__(self, state_size, action_size):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.state_size = state_size
    self.action_size = action_size #same as number_actions
    #create local Q network and target Q network
    self.local_qnetwork = Network(state_size, action_size).to(self.device)
    self.target_qnetwork = Network(state_size, action_size).to(self.device)
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate)
    self.memory = ReplayMemory(replay_buffer_size)
    self.t_step = 0

#Store experiences and decide when to learn from them
  def step(self, state, action, reward, next_state, done):
    self.memory.push((state, action, reward, next_state, done))
    self.t_step = (self.t_step + 1) % 4 #increment it, and also reset it every 4 steps
    if self.t_step == 0: #every 4 steps, it learns
    #check if no of experiences in memory is larger than minibatch size of 100, only then it learns
      if len(self.memory.memory) > minibatch_size: #self.memory is instance of replayMemory class and memory is attribute of that
        experiences = self.memory.sample(100)
        self.learn(experiences, discount_factor)


#Select action based on state baed on epsilon greedy
  def act(self, state, epsilon = 0.):
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)  # add an extra dimension corresponding to which batch this state belongs to, we add dimension of batch at beginning hence index 0
    self.local_qnetwork.eval() #set local q network in eval mode
    with torch.no_grad():
      action_values = self.local_qnetwork(state)
    self.local_qnetwork.train() #back to training mode
    if random.random() > epsilon:  #Selecting action based on epsilon
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size)) #select random number from 0,1,2,3

  #Learn method that will update agents Q values based on sample experiences
  # Update local q network weights by backpropagating, so that Q local goes towards Q target)
  # Update target q network weights by soft update
  def learn(self, experiences, discount_factor):
    states, next_states, actions, rewards, dones = experiences #unpacking
    #get max predicted Q values for next states from target network,
    #(since we're trying to compute Q predicted for the current state which includes those max predicted q values of next states in formula)
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + discount_factor * next_q_targets * (1 - dones) #Q learning formula
    q_expected = self.local_qnetwork(states).gather(1, actions) #Propogating the local q network
    loss = F.mse_loss(q_expected, q_targets)
    #back propogate the loss to update the parameters leading to better action selection
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step() #performs a single step and updates the model parameters
    self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter)  #update target network parameters with that of local network

#It will Softly update target networks parameters
  def soft_update(self, local_model, target_model, interpolation_parameter):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)

### Initializing the DQN agent

In [ ]:
agent = Agent(state_size, number_actions)

### Training the DQN agent

In [ ]:
number_episodes = 2000
maximum_number_timesteps_per_episode = 1000
epsilon_starting_value  = 1.0
epsilon_ending_value  = 0.01
epsilon_decay_value  = 0.995 #multiply by this each time
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen = 100)

for episode in range(1, number_episodes + 1):
  state, _ = env.reset()
  score = 0  #reward
  for t in range(maximum_number_timesteps_per_episode):
    action = agent.act(state, epsilon)  #at each timestep, do an action
    next_state, reward, done, _, _ = env.step(action) #to get to next step until episode is done

     #step method calls learn method which basically trains it
    agent.step(state, action, reward, next_state, done)
    score += reward
    if done:
      break
  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end = "")
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 200.0:  #condition for winning in the lunar lending, meaning we started winning from (episode - 100)th episode
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
    break

Episode 100	Average Score: -134.15
Episode 200	Average Score: -85.51
Episode 300	Average Score: 38.37
Episode 400	Average Score: 171.75
Episode 452	Average Score: 201.19
Environment solved in 352 episodes!	Average Score: 201.19


## Part 3 - Visualizing the results

In [ ]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v3')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()